Python Version of the QL Bermudan Swaption Example

In [ ]:
import QuantLib as ql

In [ ]:
todaysDate = ql.Date(15,2,2002)
calendar = ql.TARGET()
settlementDate = ql.Date(19,2,2002)
ql.Settings.instance().evaluationDate = todaysDate

flat yield term structure impling 1x5 swap at 5%

In [ ]:
flatRate = ql.SimpleQuote(0.04875825)
rhTermStructure = ql.YieldTermStructureHandle(ql.FlatForward(settlementDate,ql.QuoteHandle(flatRate),ql.Actual365Fixed()))

Define the ATM/OTM/ITM swaps

In [ ]:
fixedLegFrequency = ql.Annual
fixedLegConvention = ql.Unadjusted
floatingLegConvention = ql.ModifiedFollowing
fixedLegDayCounter = ql.Thirty360(ql.Thirty360.European)
floatingLegFrequency = ql.Semiannual
swapType = ql.VanillaSwap.Payer
dummyFixedRate = 0.03
indexSixMonths = ql.Euribor6M(rhTermStructure)

In [ ]:
startDate = calendar.advance(settlementDate, ql.Period(1,ql.Years),floatingLegConvention)
maturity = calendar.advance(settlementDate, ql.Period(5,ql.Years),floatingLegConvention)
fixedSchedule = ql.Schedule(startDate,maturity,ql.Period(fixedLegFrequency),calendar,fixedLegConvention,fixedLegConvention,
                           ql.DateGeneration.Forward,False)
floatSchedule = ql.Schedule(startDate,maturity,ql.Period(floatingLegFrequency),calendar,floatingLegConvention,floatingLegConvention,
                           ql.DateGeneration.Forward,False)

In [ ]:
swap = ql.VanillaSwap(swapType,1000.0,
            fixedSchedule, dummyFixedRate, fixedLegDayCounter,
            floatSchedule, indexSixMonths, 0.0,
            indexSixMonths.dayCounter())

In [ ]:
swap.setPricingEngine(ql.DiscountingSwapEngine(rhTermStructure))

In [ ]:
fixedATMRate = swap.fairRate()
fixedOTMRate = fixedATMRate * 1.2
fixedITMRate = fixedATMRate * 0.8

In [ ]:
atmSwap = ql.VanillaSwap(swapType,1000.0,
            fixedSchedule, fixedATMRate, fixedLegDayCounter,
            floatSchedule, indexSixMonths, 0.0,
            indexSixMonths.dayCounter())
otmSwap = ql.VanillaSwap(swapType,1000.0,
            fixedSchedule, fixedOTMRate, fixedLegDayCounter,
            floatSchedule, indexSixMonths, 0.0,
            indexSixMonths.dayCounter())
itmSwap = ql.VanillaSwap(swapType,1000.0,
            fixedSchedule, fixedITMRate, fixedLegDayCounter,
            floatSchedule, indexSixMonths, 0.0,
            indexSixMonths.dayCounter())

In [ ]:
swaptionMaturities = [ql.Period(i, ql.Years) for i in range(1,6)]

In [ ]:
numRows = 5
numCols = 5
swapLenghts = [      1,     2,     3,     4,     5]
swaptionVols = [
  0.1490, 0.1340, 0.1228, 0.1189, 0.1148,
  0.1290, 0.1201, 0.1146, 0.1108, 0.1040,
  0.1149, 0.1112, 0.1070, 0.1010, 0.0957,
  0.1047, 0.1021, 0.0980, 0.0951, 0.1270,
  0.1000, 0.0950, 0.0900, 0.1230, 0.1160]

In [ ]:
swaptions = []
#List of times that have to be included in the timegrid
times = []
for i in range(0,numRows):
    j = numCols - i -1 #1x5, 2x4, 3x3, 4x2, 5x1
    k = i*numCols + j
    vol = ql.SimpleQuote(swaptionVols[k])
    swaptions += [ql.SwaptionHelper(swaptionMaturities[i],
                               ql.Period(swapLenghts[j], ql.Years),
                               ql.QuoteHandle(vol),
                               indexSixMonths,
                               indexSixMonths.tenor(),
                               indexSixMonths.dayCounter(),
                               indexSixMonths.dayCounter(),
                               rhTermStructure)]
    times+=[t for t in swaptions[-1].times()]

Building time-grid

In [ ]:
grid = ql.TimeGrid(times, 30)

defining the models

In [ ]:
modelG2 = ql.G2(rhTermStructure)
modelHW = ql.HullWhite(rhTermStructure)
modelHW2 = ql.HullWhite(rhTermStructure)
modelBK = ql.BlackKarasinski(rhTermStructure)

model calibrations

In [ ]:
print("G2 (analytic formulae) calibration")
for s in swaptions:
    s.setPricingEngine(ql.G2SwaptionEngine(modelG2, 6.0, 16))

In [ ]:
def calibrateModel(model, helpers):
    om = ql.LevenbergMarquardt()
    model.calibrate(helpers, om, ql.EndCriteria(400, 100, 1.0e-8, 1.0e-8, 1.0e-8))
    for i in range(0,numRows):
        j = numCols - i -1 # 1x5, 2x4, 3x3, 4x2, 5x1
        k = i*numCols + j;
        npv = helpers[i].modelValue()
        implied = helpers[i].impliedVolatility(npv, 1e-4, 1000, 0.05, 0.50)
        diff = implied - swaptionVols[k]
        print('{:<1}{:<1}{:<1}: model {:<7}, market {:<7} ({:<7})'.format(str(i+1), ("x"), str(swapLenghts[j]),("%.6f"%implied),
                                                                 ("%.6f"%swaptionVols[k]),("%.6f"%diff)))

In [ ]:
calibrateModel(modelG2, swaptions)

In [ ]:
print( "calibrated to:")
print( "a     =" ,"%.6f"%modelG2.params()[0])
print( "sigma =" ,"%.6f"%modelG2.params()[1])
print( "b     =" ,"%.6f"%modelG2.params()[2])
print( "eta   =" ,"%.6f"%modelG2.params()[3])
print( "rho   =" ,"%.6f"%modelG2.params()[4])

In [ ]:
print("Hull-White (analytic formulae) calibration")

In [ ]:
for s in swaptions:
    s.setPricingEngine(ql.JamshidianSwaptionEngine(modelHW))

In [ ]:
calibrateModel(modelHW, swaptions)

In [ ]:
print( "calibrated to:")
print( "a     =" ,"%.6f"%modelHW.params()[0])
print( "sigma =" ,"%.6f"%modelHW.params()[1])

In [ ]:
print("Hull-White (numerical) calibration")

In [ ]:
for s in swaptions:
    s.setPricingEngine(ql.TreeSwaptionEngine(modelHW2, grid))

In [ ]:
calibrateModel(modelHW2, swaptions)

In [ ]:
print( "calibrated to:")
print( "a     =" ,"%.6f"%modelHW2.params()[0])
print( "sigma =" ,"%.6f"%modelHW2.params()[1])

In [ ]:
print("Black-Karasinski (numerical) calibration")

In [ ]:
for s in swaptions:
    s.setPricingEngine(ql.TreeSwaptionEngine(modelBK, grid))

In [ ]:
calibrateModel(modelBK, swaptions)

In [ ]:
print( "calibrated to:")
print( "a     =" ,"%.6f"%modelBK.params()[0])
print( "sigma =" ,"%.6f"%modelBK.params()[1])

ATM Bermudan swaption pricing

In [ ]:
print("Payer bermudan swaption struck at %.6f (ATM)" % fixedATMRate)

In [ ]:
leg = swap.fixedLeg()

In [ ]:
bermudanDates = [d for d in fixedSchedule]

In [ ]:
bermudanExercise = ql.BermudanExercise(bermudanDates)

In [ ]:
bermudanSwaption = ql.Swaption(atmSwap, bermudanExercise)

Do the pricing for each model

G2 price the European swaption here, it should switch to bermudan

In [ ]:
def calc_swaption_npv (swaption):
    try:
        swaption.setPricingEngine(ql.TreeSwaptionEngine(modelG2, 50))
        print ("G2 (tree):      %.6f" % swaption.NPV())
    except:
        pass
    try:
        swaption.setPricingEngine(ql.FdG2SwaptionEngine(modelG2))
        print ("G2 (fdm):      %.6f" % swaption.NPV())
    except:
        pass
    try:
        swaption.setPricingEngine(ql.TreeSwaptionEngine(modelHW, 50))
        print ("HW (tree):      %.6f" % swaption.NPV())
    except:
        pass
    try:
        swaption.setPricingEngine(ql.FdHullWhiteSwaptionEngine(modelHW))
        print ("HW (fdm):      %.6f" % swaption.NPV())
    except:
        pass
    try:
        swaption.setPricingEngine(ql.TreeSwaptionEngine(modelHW2, 50))
        print ("HW (num, tree): %.6f" % swaption.NPV())
    except:
        pass
    try:
        swaption.setPricingEngine(ql.FdHullWhiteSwaptionEngine(modelHW2))
        print ("HW (num, fdm):      %.6f" % swaption.NPV())
    except:
        pass
    try:
        swaption.setPricingEngine(ql.TreeSwaptionEngine(modelBK, 50))
        print ("BK:             %.6f" % swaption.NPV())
    except:
        pass

In [ ]:
calc_swaption_npv(bermudanSwaption)

OTM Bermudan swaption pricing

In [ ]:
print("Payer bermudan swaption struck at %.6f (OTM)" % fixedOTMRate)

In [ ]:
otmBermudanSwaption =ql.Swaption(itmSwap,bermudanExercise)

In [ ]:
calc_swaption_npv(otmBermudanSwaption)

ITM Bermudan swaption pricing

In [ ]:
print("Payer bermudan swaption struck at %.6f (ITM)" % fixedITMRate)

In [ ]:
itmBermudanSwaption =ql.Swaption(itmSwap,bermudanExercise)

In [ ]:
calc_swaption_npv(itmBermudanSwaption)